# Brusselator model

The dynamics of the oscillating reaction discovered by Belousov and Zhabotinsky, 
can be modeled through the so-called Brusselator model depending on two parameters:

$$
\left\{\begin{aligned}
\frac{dy_1}{dt} & = 1 - (b+1) y_1 + a y_1^2y_2\\
\frac{dy_2}{dt} & = b y_1 - a y_1^2y_2
\end{aligned}\right.
$$

In [1]:
import numpy as np

from scipy.integrate import solve_ivp

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.models import PrintfTickFormatter

from ipywidgets import interact, IntSlider, Dropdown

from mylib.model import brusselator_model
import mylib.integration as integration

output_notebook(hide_banner=True)

## Quasi-exact solution

The quasi-exact solution is obtained by using an explicit Runge-Kutta method of order 5 with stepsize control due to Dormand and Prince.

In [2]:
def plot_quasi_exact_sol():
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    
    tini = 0. 
    tend = 20.
    
    yini = (1.5, 3)
    #yini = (4, 7)
    sol = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12)

    fig_sol = figure(x_range=(tini, tend),y_range=(0,7), width=1000, height=300, title="Solution")
    fig_sol.line(sol.t, sol.y[0],legend="y1", line_width=2)    
    fig_sol.line(sol.t, sol.y[1], legend="y2", line_width=2, color="Green")
    fig_sol.legend.location = "top_right"
    
    show(fig_sol)
    
plot_quasi_exact_sol()

stiff就是变化速率

## Characterisation of stiffness

In [3]:
def plot_eigenvalue():

    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    jac = bm.jac
    
    tini = 0. 
    tend = 20.
    
    yini = (1.5 , 3)
    
    sol = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12)
    
    eig_vals = np.zeros((sol.t.size, 2), dtype=np.complex_)
    for it in range(0,sol.t.size):
        eig_vals[it], _ = np.linalg.eig(jac(sol.y[:,it]))

    lambda1 = eig_vals[:, 0]
    lambda2 = eig_vals[:, 1]
    
    fig_real = figure(x_range=(tini, tend), plot_height=300, plot_width=900, 
                      title = "Real part of eigenvalues (click on legend entries to hide corresponding plot)")
    fig_imag = figure(x_range=(tini, tend), plot_height=300, plot_width=900, 
                      title = "Imaginary part of eigenvalues (click on lengend to hide corresponding plot)")
            
    fig_real.line(sol.t, np.real(lambda1), line_width=2, legend="lambda1")
    fig_real.line(sol.t, np.real(lambda2), line_width=2, color="Green", legend="lambda2")
    fig_real.legend.click_policy="hide"

    fig_imag.line(sol.t, np.imag(lambda1), line_width=2, legend="lambda1")
    fig_imag.line(sol.t, np.imag(lambda2), line_width=2, color="Green", legend="lambda2")
    fig_imag.legend.click_policy="hide"

    show(column(fig_real, fig_imag))
    
plot_eigenvalue()

* 当特征值为正的时候，我们也无法判断是否stable，因为本身系统就已经不stable了
* 当implicit的时候


## Runge-Kutta methods

In [4]:
def plot_rk_methods():

    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    
    tini = 0.
    tend = 50.
    
    yini = (1.5 , 3)
    
    fig_sol = figure(x_range=(tini, tend), y_range=(0, 10),plot_height=300, plot_width=700, title="Solution")
    fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=700, title="Global error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")
    
    nt =int(60*tend)
    t = np.linspace(tini, tend, nt)
    
    sol = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12, t_eval=t)
    
    y = integration.forward_euler(tini, tend, nt, yini, fcn)
    
    y1_err = np.abs(sol.y[0] - y[:, 0])
    y2_err = np.abs(sol.y[1] - y[:, 1])
    
    # plot exact and rk solutions 
    plt_sol_y1 = fig_sol.x(t, y=y[:, 0], line_width=2, legend="y1")
    plt_exa_y1 = fig_sol.line(t, sol.y[0], color="Grey")
    plt_sol_y2 = fig_sol.x(t, y=y[:, 1], line_width=2, color="Green", legend="y2")
    plt_exa_y1 = fig_sol.line(t, sol.y[1], color="Grey")

    plt_err_y1 = fig_err.x(t, y1_err, line_width=2, legend="y1")
    plt_err_y2 = fig_err.x(t, y2_err, line_width=2, color="Green", legend="y2")
    
    fig_err.legend.location = "top_left"
    fig_sol.legend.location = "top_left"
    
    show(column(fig_sol, fig_err), notebook_handle=True)
    
    def update(method, nt):
        t = np.linspace(tini, tend, nt)
        if method == "rk1":
            y = integration.forward_euler(tini, tend, nt, yini, fcn)
        elif method == "rk2":
            y = integration.rk2(tini, tend, nt, yini, fcn)
        elif method == "rk3":
            y= integration.rk3(tini, tend, nt, yini, fcn)
        elif method == "rk4":
            y = integration.rk4(tini, tend, nt, yini, fcn)        

        sol = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12, t_eval=t)
        y1_err = np.abs(sol.y[0] - y[:, 0])
        y2_err = np.abs(sol.y[1] - y[:, 1])
        
        plt_sol_y1.data_source.data = dict(x=t, y=y[:, 0])
        plt_sol_y2.data_source.data = dict(x=t, y=y[:, 1])
        plt_err_y1.data_source.data = dict(x=t, y=y1_err)
        plt_err_y2.data_source.data = dict(x=t, y=y2_err)
        push_notebook()
        
    interact(update,
             nt=IntSlider(min=100, max=nt, value=nt, step=100, continuous_update=False),
             method=Dropdown(options=["rk1", "rk2", "rk3", "rk4"], value='rk1', description='Method:'))


plot_rk_methods()

interactive(children=(Dropdown(description='Method:', options=('rk1', 'rk2', 'rk3', 'rk4'), value='rk1'), IntS…

* 误差来源，stiffness， stablilty，如果是conservative的系统，error就直接跑了，但是这里是dissipative system，误差虽然上去了，但是后来还是回来。

In [5]:
def show_fe_order():
    k=50
   
    tini = 0.
    tend = 10
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn
    yini = (1.5 , 3)
    l_nt = range(1001,50001,1000)

    err1 = np.zeros((len(l_nt),2))
    err2 = np.zeros((len(l_nt),2))
    err3 = np.zeros((len(l_nt),2))
    err4 = np.zeros((len(l_nt),2))
    for i in range(0, len(l_nt)):

        i_nt = l_nt[i]
        t = np.linspace(tini, tend, i_nt)
        sol = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-13, atol=1.e-13, t_eval=t)
        
        tnum = np.linspace(tini, tend, i_nt)
        unum1 = integration.forward_euler(tini, tend, i_nt, yini, fcn)
        unum2 = integration.rk2(tini, tend, i_nt, yini, fcn)
        unum3 = integration.rk3(tini, tend, i_nt, yini, fcn)
        unum4 = integration.rk4(tini, tend, i_nt, yini, fcn)
        

        yerr = [np.abs(sol.y[0] - unum1[:, 0]),np.abs(sol.y[1] - unum1[:, 1])]
        norm_err = np.linalg.norm(yerr) / np.sqrt(l_nt[i])   
        

        err1[i] = [(tend-tini)/(i_nt-1),norm_err] 
        yerr = [np.abs(sol.y[0] - unum2[:, 0]),np.abs(sol.y[1] - unum2[:, 1])]
        norm_err = np.linalg.norm(yerr) / np.sqrt(l_nt[i])  
        err2[i] = [(tend-tini)/(i_nt-1), norm_err]
        yerr = [np.abs(sol.y[0] - unum3[:, 0]),np.abs(sol.y[1] - unum3[:, 1])]
        norm_err = np.linalg.norm(yerr) / np.sqrt(l_nt[i])
        err3[i] = [(tend-tini)/(i_nt-1), norm_err] 
        yerr = [np.abs(sol.y[0] - unum4[:, 0]),np.abs(sol.y[1] - unum4[:, 1])]
        norm_err = np.linalg.norm(yerr) / np.sqrt(l_nt[i])
        err4[i] = [(tend-tini)/(i_nt-1), norm_err]
        
    ref = np.array([1.e-1, 1.e-2, 1.e-3, 1.e-4, 1.e-5, 1.e-7 ])

    f = figure(x_axis_type="log", y_axis_type="log", title="Error", x_range=[5.e-5, 5.e-2], 
               y_range=[5.e-14, 5.e-1],plot_height=400,plot_width=550)

    f.line(ref, ref, line_width=2, legend="slope 1", color="black", line_dash='dotted')
    f.line(ref, ref*ref, line_width=2, legend="slope 2", color="brown", line_dash='dotted')
    f.line(ref, ref**3, line_width=2, legend="slope 3", color="red", line_dash='dotted')
    f.line(ref, ref**4, line_width=2, legend="slope 4", color="purple", line_dash='dotted')
    f.x(err1[:, 0], err1[:, 1], size=10, line_width=2, legend="RK1")
    f.x(err2[:, 0], err2[:, 1], size=10, line_width=2, color="crimson", legend="RK2")
    f.x(err3[:, 0], err3[:, 1], size=10, line_width=2, color="green", legend="RK3")
    f.x(err4[:, 0], err4[:, 1], size=10, line_width=2, color="black", legend="RK4")
    f.legend.location = "top_left"
    f.legend.click_policy="hide"

    show(f)
    
a=show_fe_order()

### Computational cost

In [6]:
def plot_rk_cost():

    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    
    tini = 0. 
    tend = 20.
    
    yini = (1.5 , 3)
    
    l_nt = [101, 1001, 10001, 100001]
    fe_rk1=[]
    norm_rk1=[]
    fe_rk2=[]
    norm_rk2=[]
    fe_rk3=[]
    norm_rk3=[]
    fe_rk4=[]
    norm_rk4=[]
    
    for nt in l_nt:
        
        sol = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12, t_eval=np.linspace(tini,tend,nt))
        yexa = np.transpose(sol.y)
       
        yrk1 = integration.forward_euler(tini, tend, nt, yini, fcn)
        fe_rk1.append(nt-1)
        norm_rk1.append(np.linalg.norm(yexa-yrk1) / np.sqrt(2*nt))

        yrk2 = integration.rk2(tini, tend, nt, yini, fcn)
        fe_rk2.append(2*(nt-1))
        norm_rk2.append(np.linalg.norm(yexa-yrk2) / np.sqrt(2*nt))
        
        yrk3 = integration.rk3(tini, tend, nt, yini, fcn)
        fe_rk3.append(3*(nt-1))
        norm_rk3.append(np.linalg.norm(yexa-yrk3) / np.sqrt(2*nt))
        
        if (nt == 100001):
            nt_rk4 = 20001
            sol = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12, t_eval=np.linspace(tini,tend,nt_rk4))
            yexa = np.transpose(sol.y)
            yrk4 = integration.rk4(tini, tend, nt_rk4, yini, fcn)
            fe_rk4.append(4*(nt_rk4-1))
            norm_rk4.append(np.linalg.norm(yexa-yrk4) / np.sqrt(2*nt_rk4))
        else:
            yrk4 = integration.rk4(tini, tend, nt, yini, fcn)
            fe_rk4.append(4*(nt-1))
            norm_rk4.append(np.linalg.norm(yexa-yrk4) / np.sqrt(2*nt))

        
    fig = figure(x_axis_type="log", y_axis_type="log", plot_height=400, plot_width=550,
                 title= "Number of function evaluations vs norm of global error" )
    
    fig.x(norm_rk1, fe_rk1, legend="rk1")
    fig.line(norm_rk1, fe_rk1, legend="rk1")
    fig.x(norm_rk2, fe_rk2, legend="rk2", color="Green")
    fig.line(norm_rk2, fe_rk2, legend="rk2", color="Green")
    fig.x(norm_rk3, fe_rk3, legend="rk3", color="Red")
    fig.line(norm_rk3, fe_rk3, legend="rk3", color="Red")
    fig.x(norm_rk4, fe_rk4, legend="rk4", color="Indigo")
    fig.line(norm_rk4, fe_rk4, legend="rk4", color="Indigo")
    show(fig)
    
    
plot_rk_cost()

横坐标是误差，左边是调用、循环次数
可以比较，如果我确定纵坐标，然后我看不同的方法给的误差

### Implicit Euler

In [7]:
def plot_implicit_euler_sol():
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    jac = bm.jac
    
    tini = 0. 
    tend = 100.
    #可以玩nt，原来是20
    nt = 3000
    t = np.linspace(tini, tend, nt)
    dt = (tend-tini)/(nt-1)
    
    yini = (1.5 , 3)
    
    sol = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12)
    y = integration.backward_euler(tini, tend, nt, yini, fcn)
    
    sol_eig = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12, t_eval=t)
    fcn_stab = np.zeros((t.size, 2))
    real_eig_vals = np.zeros((t.size))
    for it in range(0,t.size):
        eig_val, _ = np.linalg.eig(jac(sol_eig.y[:,it]))
        real_eig_vals[it] = np.real(eig_val)[0]
        fcn_stab[it] = np.absolute(1-dt*eig_val)
        
    mask = np.where(real_eig_vals>0)
    fcn_stab_1 = fcn_stab[:,0]
    fcn_stab_2 = fcn_stab[:,0]

    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    fig_sol.line(sol.t, sol.y[0], color="black", line_dash="dotted")    
    fig_sol.line(sol.t, sol.y[1], color="black", line_dash="dotted")    
    fig_sol.x(t, y[:,0], legend="y1", line_width=2)    
    fig_sol.x(t, y[:,1], legend="y2", line_width=2, color="Green")
    fig_sol.legend.location = "top_left"
    
    fig_stab = figure(x_range=(tini, tend), width=950, height=300, title="Value of | 1 - z |")
    fig_stab.line(t, np.ones(t.size), color="black", line_dash="dotted")
    fig_stab.x(t, fcn_stab_1, line_width=2, legend='Value of  | 1-z | for negative eigen values')
    fig_stab.x(t, fcn_stab_2, line_width=2)
    fig_stab.x(t[mask], fcn_stab_1[mask], line_width=2, color="crimson", legend='Value of  | 1-z | for positive eigen values')    
    fig_stab.x(t[mask], fcn_stab_2[mask], line_width=2, color="crimson")
    fig_stab.legend.background_fill_alpha = 0.5
        
    show(column(fig_sol, fig_stab))

plot_implicit_euler_sol()

* implicit euler, A-stable,
* 蓝色的是负的特征值，主要要看红色的点，因为这个时候numerically 还是stable的，但是实际上已经不行了。正的特征值，但是scheme还是stable，有问题
* 我们可以看到，system contractant， croissance explosion
* 第二张图如果超过一，那么就在那个stable范围之内，但是事实上，我们发现，数值解（蓝色的部分）并不能捕捉到stiff，那个快速增长的地方
* 如果增加$n_t$,那么红色的点基本上不在1之上，从而保证两个都在快速增长。
* stablite A类型和B类型， L类型，都是在contractant的情况中，也就是$Re(\lambda)<0$

In [8]:
def plot_rk38_sol():
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    
    tini = 0. 
    tend = 20.
    nt = 200
    tnum = np.linspace(tini, tend, nt)
    
    yini = (1.5 , 3)
    
    tol = 1.e-12
    sol = solve_ivp(fcn, (tini, tend), yini, rtol=tol, atol=tol)
    
    yrk38 = integration.rk38(tini, tend, nt, yini, fcn)

    fig_sol = figure(x_range=(tini, tend), width=950, height=300)
    fig_sol.line(sol.t, sol.y[0], color="Grey")
    fig_sol.line(sol.t, sol.y[1], color="Grey")
    fig_sol.x(tnum, yrk38[:, 0], legend="y1", line_width=2)    
    fig_sol.x(tnum, yrk38[:, 1], legend="y2", line_width=2, color="Green")
    fig_sol.legend.location = "top_left"
    
    show(fig_sol)
    
plot_rk38_sol()

## Enbedded Runge-Kutta methods

### Method based on 3/8 fourth-order Runge-Kutta method

In [9]:
def plot_rk43_embedded_sol():
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    
    tini = 0. 
    tend = 20.
    nt = 200
    
    yini = (1.5 , 3)
    
    sol = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12)

    tol = 1.e-4
    sol_rk43 = integration.rk43_embedded(tini, tend, nt, yini, fcn, tol)
    
    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    fig_sol.line(sol.t, sol.y[0], color="Grey")
    fig_sol.line(sol.t, sol.y[1], color="Grey")
    plt_y1 = fig_sol.x(sol_rk43.t, sol_rk43.y[:, 0], legend="y1", line_width=2)    
    plt_y2 = fig_sol.x(sol_rk43.t, sol_rk43.y[:, 1], legend="y2", line_width=2, color="Green")
    fig_sol.legend.location = "top_left"
  
    fig_dt = figure(x_range=(tini, tend), plot_height=300, plot_width=950, y_axis_type="log", 
                    title = "Accepted and rejcted times step") 
    plt_dt = fig_dt.square(sol_rk43.t[0:-1], sol_rk43.dt, color="Black")
    plt_line_dt = fig_dt.line(sol_rk43.t[0:-1], sol_rk43.dt, color="Black")
    plt_dt_rej = fig_dt.circle_x(sol_rk43.t_rej, sol_rk43.dt_rej, color="Crimson")

    fig_err = figure(x_range=(tini, tend), y_axis_type="log", width=950, height=300, title="Error")
    plt_loc_err = fig_err.x(sol_rk43.t, sol_rk43.loc_err, legend="estimate local error", line_width=2)
    plt_loc_err_exa = fig_err.circle(sol_rk43.t, sol_rk43.loc_err_exa, legend="local error", fill_alpha=0., color="crimson")
    plt_glob_err = fig_err.square(sol_rk43.t, sol_rk43.glob_err, legend="global error", fill_alpha=0, color="green")
    plt_tol = fig_err.line(sol_rk43.t, tol*np.ones(sol_rk43.t.size), color="grey", line_dash="dotted")    
    fig_err.legend.location = "bottom_left"
    fig_err.legend.orientation = "horizontal"

    show(column(fig_sol, fig_dt, fig_err), notebook_handle=True)
    
    def update(tol):
        
        sol_rk43 = integration.rk43_embedded(tini, tend, nt, yini, fcn, tol)

        plt_y1.data_source.data = dict(x=sol_rk43.t, y=sol_rk43.y[:, 0])
        plt_y2.data_source.data = dict(x=sol_rk43.t, y=sol_rk43.y[:, 1]) 

        plt_dt.data_source.data = dict(x=sol_rk43.t[0:-1], y=sol_rk43.dt)
        plt_line_dt.data_source.data = dict(x=sol_rk43.t[0:-1], y=sol_rk43.dt) 
        plt_dt_rej.data_source.data = dict(x=sol_rk43.t_rej, y=sol_rk43.dt_rej)
        
        plt_loc_err.data_source.data = dict(x=sol_rk43.t, y=sol_rk43.loc_err)
        plt_loc_err_exa.data_source.data  = dict(x=sol_rk43.t, y=sol_rk43.loc_err_exa)
        plt_glob_err.data_source.data = dict(x=sol_rk43.t, y=sol_rk43.glob_err)
        plt_tol.data_source.data = dict(x=sol_rk43.t, y=tol*np.ones(sol_rk43.t.size))

        push_notebook()
        
        
    dtol={'1.e-1':1.e-1, '1.e-2':1.e-2, '1.e-3':1.e-3, '1.e-4':1.e-4, '1.e-5':1.e-5, '1.e-6':1.e-6}     
    interact(update, tol=Dropdown(options=dtol, value=1.e-4, description='tol'))

    
plot_rk43_embedded_sol()

interactive(children=(Dropdown(description='tol', index=3, options={'1.e-1': 0.1, '1.e-2': 0.01, '1.e-3': 0.00…

### Computational cost

In [10]:
def plot_embedded_cost():

    yini = (1.5 , 3.)
    tini = 0.
    tend = 20.
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn
            
    l_nt = [101, 501, 1001, 5001, 10001, 20001]
    fe_rk38=[]
    norm_rk38=[]
    
    for nt in l_nt:
        sol = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12, t_eval=np.linspace(tini, tend, nt))
        yexa = np.transpose(sol.y)
        yrk38 = integration.rk38(tini, tend, nt, yini, fcn)
        fe_rk38.append(4*(nt-1))
        norm_rk38.append(np.linalg.norm(yexa-yrk38) / np.sqrt(2*nt))
        
              
    l_tol = [1.e-2, 1.e-3, 1.e-4, 1.e-5, 1.e-6, 1.e-7, 1.e-9, 1.e-10, 1.e-12]
    fe_rk43_emb=[]
    norm_rk43_emb=[]
    
    for tol in l_tol:
        #print("tol =", tol)
        sol_rk43_emb = integration.rk43_embedded(tini, tend, 1000, yini, fcn, tol)
        sol = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12, t_eval=sol_rk43_emb.t)
        fe_rk43_emb.append(4*sol_rk43_emb.dt.size + 4*sol_rk43_emb.dt_rej.size)
        err = 0 
        for i in range(sol_rk43_emb.dt.size):
            ldt = sol_rk43_emb.dt[i]/(tend-tini)
            err += ldt * ((sol.y[0,i+1]-sol_rk43_emb.y[i+1,0])*(sol.y[0,i+1]-sol_rk43_emb.y[i+1,0]) + \
                          (sol.y[1,i+1]-sol_rk43_emb.y[i+1,1])*(sol.y[1,i+1]-sol_rk43_emb.y[i+1,1])  )
        err = np.sqrt(err)    
        norm_rk43_emb.append(err)
    

    fig = figure(x_axis_type="log", y_axis_type="log", plot_height=500, plot_width=900, \
                 title= "Number of function evaluations vs norm of global error" )
    fig.x(norm_rk38, fe_rk38, legend="rk38", color="Indigo")
    fig.line(norm_rk38, fe_rk38, legend="rk38", color="Indigo")
    fig.x(norm_rk43_emb, fe_rk43_emb, legend="rk embedded", color="Green")
    fig.line(norm_rk43_emb, fe_rk43_emb, legend="rk embedded", color="Green")

    show(fig)
    
    
plot_embedded_cost()

* embedded给出了一个更好的解决方案
* 

## Dormand and Price method

In [11]:
def plot_dopri5_sol():
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    
    tini = 0. 
    tend = 20.
    
    yini = (1.5, 3)
    
    fig_sol = figure(x_range=(tini, tend),  plot_height=300, plot_width=900, title="Solution")
    fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=900, title="Global error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")

    tol = 1.e-4
    sol_dopri5 = solve_ivp(fcn, (tini, tend), yini, rtol=tol, atol=tol)
    sol = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12, t_eval=sol_dopri5.t)
       
    y1_err = np.abs(sol.y[0] - sol_dopri5.y[0])
    y2_err = np.abs(sol.y[1] - sol_dopri5.y[1])

    plt_sol_y1 = fig_sol.x(sol_dopri5.t, sol.y[0], legend="y1", line_width=2)    
    plt_sol_y2 = fig_sol.x(sol_dopri5.t, sol.y[1], legend="y2", line_width=2, color="Green")
    fig_sol.legend.location = "top_left"
    
    plt_err_y1 = fig_err.x(sol_dopri5.t, y1_err, line_width=2, legend="y1")
    plt_err_y2 = fig_err.x(sol_dopri5.t, y2_err, line_width=2, color="Green", legend="y2")
    
    show(column(fig_sol, fig_err), notebook_handle=True)
    
    def update(tol):
        
        sol_dopri5 = solve_ivp(fcn, (tini, tend), yini, rtol=tol, atol=tol)
        sol = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12, t_eval=sol_dopri5.t)
        
        y1_err = np.abs(sol.y[0] - sol_dopri5.y[0])
        y2_err = np.abs(sol.y[1] - sol_dopri5.y[1])


        plt_sol_y1.data_source.data = dict(x=sol_dopri5.t, y=sol_dopri5.y[0])
        plt_sol_y2.data_source.data = dict(x=sol_dopri5.t, y=sol_dopri5.y[1]) 
        plt_err_y1.data_source.data = dict(x=sol_dopri5.t, y=y1_err)
        plt_err_y2.data_source.data = dict(x=sol_dopri5.t, y=y2_err) 


        push_notebook()
        
    dtol={'1.e-2':1.e-2, '1.e-4':1.e-4, '1.e-6':1.e-6, '1.e-8':1.e-8, '1.e-10':1.e-10}     
    interact(update, tol=Dropdown(options=dtol, value=1.e-4, description='tol'))

    
plot_dopri5_sol()

interactive(children=(Dropdown(description='tol', index=1, options={'1.e-2': 0.01, '1.e-4': 0.0001, '1.e-6': 1…

* 比较慢，但是相对来讲比较准确